In [1]:
import fitz  # pip install pymupdf

import numpy as np
from shapely.geometry import box
import pandas as pd

import io

from shapely.geometry import box
from shapely.ops import unary_union
import os
from collections import defaultdict
from tqdm.notebook import tqdm

import pytesseract
from PIL import Image, ImageEnhance, ImageFilter
from collections import defaultdict
from tqdm.notebook import tqdm

import pickle

import numpy as np

import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction

from chromadb.utils.data_loaders import ImageLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_experimental.open_clip import OpenCLIPEmbeddings

from operator import itemgetter

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

In [ ]:
class IngestionPipeline:
    
    def __init__(self, dir_path: str, create=True):
        self.dir_path = dir_path
        
        # set default text and image embedding functions
        self.embedding_function = OpenCLIPEmbeddingFunction()
        self.chroma_collection = self.create_vector_store(create=create)

    
    def create_vector_store(self, create=True, db_dir='chromadb/'):

        db_name = self.dir_path + db_dir
        if not os.path.exists(db_name):
            print("Creating vector store: ", db_name)
            os.mkdir(db_name)
        else:
            print("Updating vector store:", db_name)

        # create client and a new collection
        self.chroma_client = chromadb.PersistentClient(path=db_name, settings=Settings(allow_reset=True))
        image_loader = ImageLoader()

        if create:
            print("Deleting existing collection ...")
            self.chroma_client.reset()
            print("Creating new collection ...")
            chroma_collection = self.chroma_client.get_or_create_collection(
                "multimodal_collection", metadata={"hnsw:space": "cosine"},
                embedding_function=self.embedding_function,
                data_loader=image_loader
            )
        else:
            print("Recovering existing collection ...")
            chroma_collection = self.chroma_client.get_or_create_collection(
                "multimodal_collection", metadata={"hnsw:space": "cosine"},
                embedding_function=self.embedding_function,
                data_loader=image_loader
            )
            print("elements:", chroma_collection.count())

        return chroma_collection

    @staticmethod
    def are_rectangles_connected(rect1, rect2, threshold=0):
        """
        Check if two rectangles are connected (overlap or close within a threshold).
        
        :param rect1: Tuple (x1, y1, x2, y2) for the first rectangle.
        :param rect2: Tuple (x1, y1, x2, y2) for the second rectangle.
        :param threshold: Minimum distance to consider rectangles as connected.
        :return: True if connected, False otherwise.
        """
        b1 = box(*rect1).buffer(threshold)  # Expand rect1 by threshold
        b2 = box(*rect2)
        return b1.intersects(b2)

    @staticmethod
    def group_connected_rectangles(rectangles, threshold=0):
        """
        Group rectangles into clusters based on connectivity.
        
        :param rectangles: List of rectangles [(x1, y1, x2, y2), ...]
        :param threshold: Distance threshold to consider rectangles connected.
        :return: List of grouped rectangles as [(grouped_x1, grouped_y1, grouped_x2, grouped_y2), ...]
        """
        n = len(rectangles)
        adjacency_matrix = np.zeros((n, n), dtype=bool)
        
        # Build adjacency matrix
        for i in range(n):
            for j in range(i + 1, n):
                if IngestionPipeline.are_rectangles_connected(rectangles[i], rectangles[j], threshold):
                    adjacency_matrix[i, j] = True
                    adjacency_matrix[j, i] = True
        
        # Find connected components
        visited = [False] * n
        groups = []
        
        def dfs(node, group):
            visited[node] = True
            group.append(node)
            for neighbor in range(n):
                if adjacency_matrix[node, neighbor] and not visited[neighbor]:
                    dfs(neighbor, group)
        
        for i in range(n):
            if not visited[i]:
                group = []
                dfs(i, group)
                groups.append(group)
        
        # Merge rectangles within each group
        grouped_rectangles = []
        for group in groups:
            x_min = min(rectangles[i][0] for i in group)
            y_min = min(rectangles[i][1] for i in group)
            x_max = max(rectangles[i][2] for i in group)
            y_max = max(rectangles[i][3] for i in group)
            grouped_rectangles.append((x_min, y_min, x_max, y_max))
        
        return grouped_rectangles
    
    @staticmethod
    def stitch_grouped_images_with_dpi(image_data, image_coords, grouped_boxes, dpi=300):
        """
        Stitch images together for each grouped bounding box, preserving size and quality.
        
        :param image_data: List of image bytes.
        :param image_coords: List of tuples [(x1, y1, x2, y2)], coordinates of the images in the PDF.
        :param grouped_boxes: List of grouped bounding boxes [(group_x1, group_y1, group_x2, group_y2)].
        :param dpi: DPI value to scale the images correctly.
        :return: List of stitched PIL.Image objects, one for each group.
        """
        # Scale factor to convert coordinates to pixels
        scale = dpi / 72  # 72 is the default DPI in PDFs
        
        stitched_images = []
        
        for group_box in grouped_boxes:
            # Scale group box dimensions to pixels
            group_x1, group_y1, group_x2, group_y2 = group_box
            canvas_width = int((group_x2 - group_x1) * scale)
            canvas_height = int((group_y2 - group_y1) * scale)
            
            # Create a blank canvas for the group
            canvas = Image.new('RGBA', (canvas_width, canvas_height), (255, 255, 255, 0))
            
            for img_bytes, coords in zip(image_data, image_coords):
                # Convert image bytes to PIL.Image
                img = Image.open(io.BytesIO(img_bytes))
                
                # Check if this image belongs to the current group
                img_x1, img_y1, img_x2, img_y2 = coords
                if (
                    group_x1 <= img_x1 < group_x2 and group_y1 <= img_y1 < group_y2
                    or group_x1 <= img_x2 <= group_x2 and group_y1 <= img_y2 <= group_y2
                ):
                    # Scale image coordinates to pixels
                    paste_x = int((img_x1 - group_x1) * scale)
                    paste_y = int((img_y1 - group_y1) * scale)
                    img_width = int((img_x2 - img_x1) * scale)
                    img_height = int((img_y2 - img_y1) * scale)
                    
                    # Resize only if necessary, using high-quality resampling
                    if img.size != (img_width, img_height):
                        img = img.resize((img_width, img_height), resample=Image.Resampling.LANCZOS)
                    
                    # Paste the image onto the canvas
                    canvas.paste(img, (paste_x, paste_y), img if img.mode == 'RGBA' else None)
            
            stitched_images.append(canvas)
    
        return stitched_images
    
    
    def _process_pdf_pages(self, pdf_filename:str, images_dir='images'):

        file = self.dir_path + pdf_filename
        print("Processing PDF pages: "+file)
        
        dict_mapping_pages = defaultdict(list)

        # open the file
        pdf_file = fitz.open(file)

        # iterate over PDF pages
        for page_index in range(len(pdf_file)):

            # get the page itself
            page = pdf_file.load_page(page_index)  # load the page
            image_list = page.get_images(full=True)  # get images on the page

            # printing number of images found in this page
            if image_list:
                print(f"[+] Found a total of {len(image_list)} images on page {page_index}")
            else:
                print("[!] No images found on page", page_index)
            
            image_data = []
            image_coords = []
            for image_index, img in enumerate(image_list, start=1):
                # get the XREF of the image
                xref = img[0]

                # extract the image bytes
                base_image = pdf_file.extract_image(xref)
                image_bytes = base_image["image"]

                # get the image extension
                image_ext = base_image["ext"]

                img_rect = page.get_image_rects(xref)[0]
                coords = (img_rect.x0, img_rect.y0, img_rect.x1, img_rect.y1)

                # print(page_index+1,image_index,coords)

                image_data.append(image_bytes)
                image_coords.append(coords)

            grouped_boxes = IngestionPipeline.group_connected_rectangles(image_coords, threshold=0)

            # print(grouped_boxes)

            stitched_images = IngestionPipeline.stitch_grouped_images_with_dpi(image_data, image_coords, grouped_boxes)

            # Save or display the stitched images

            if not os.path.exists(self.dir_path + images_dir):
                os.mkdir(self.dir_path + images_dir)

            for i, img in enumerate(stitched_images):
                fname =  self.dir_path+images_dir+'/stitched_group_'+str(page_index+1)+'_'+str(i)+'.png'
                print('Saving... ', fname)
                img.save(fname)
                dict_mapping_pages[page_index+1].append(f'stitched_group_{page_index+1}_{i}')
            
        return dict_mapping_pages
    
    @staticmethod
    def load_image(infilename): # Read image from file system and convert it to numpy array
        img = Image.open( infilename )
        img.load()
        data = np.asarray(img, dtype="int32")
        return data

    def _extract_paragraphs(self, pdf_filename:str, header_height=140, footer_height=158):
        file = self.dir_path + pdf_filename

        print("Extracting paragraphs: "+file)
        doc = fitz.open(file)

        dict_extracted_paragraphs = defaultdict(list)

        # header_height = 140 # manually defined
        # footer_height = 158

        # Iterate over the pages of the PDF
        for page_num in tqdm(range(len(doc)), "pages"):
        # for page_num in range(len(doc)):

            # print("-"*10, page_num)

            page = doc.load_page(page_num)  # Load the page
            
            # Convert the page to an image (pixmap)
            pix = page.get_pixmap()

            # Increase resolution by scaling the page (2x in this case)
            matrix = fitz.Matrix(2, 2)  # Scaling by a factor of 2
            pix = page.get_pixmap(matrix=matrix)  # Convert page to high-res image

            # Convert to Pillow Image
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

            width, height = img.size
            img = img.crop((0, header_height, width, height - footer_height))
            
            # Apply Tesseract OCR to the image
            text = pytesseract.image_to_string(img)

            # Print out the extracted text
            # print(f"Text from page {page_num + 1}:\n{text}\n")
            dict_extracted_paragraphs[page_num + 1].append(text.replace('\n',''))
            # print('='*10)
        
        return dict_extracted_paragraphs
    
    def _ingest_images(self, images_dir):
        print("Adding images to vector store ...")
        for page,images in self.dict_mapping_pages.items():
            for im in images:
                image_name = self.dir_path + images_dir + im + '.png' # TODO: Fix name
                # print(image_name, os.path.exists(image_name))
                if os.path.exists(image_name): 
                    print(image_name)
                    
                    # NOTE: si o si hay que agregar el documents porque sino la class Document explota cuando hacemos
                    # busquedas porque falta ese field.
                    # En algun momento se guardo el base64 de la image, pero no tiene mucho sentido ya que con el uris
                    # podemos recuperar la imagen original si es necesario.
                    
                    self.chroma_collection.add(ids=[im],
                                        uris=[image_name],
                                        metadatas=[{'page':page,'type':'image'}])
    
    def _ingest_text(self):
        print("Adding text to vector store ...")
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
        for page,paragraphs in self.dict_extracted_paragraphs.items():
            total_chunks = 0
            for i in range(0,len(paragraphs)):
                chunked_documents = text_splitter.split_text(paragraphs[i]) 
                total_chunks += len(chunked_documents)
                for j in range(0,len(chunked_documents)):
                    self.chroma_collection.add(
                        ids=[f'paragraph_{page}_{i}_{j}'],
                            documents= [chunked_documents[j]],
                            metadatas = [{'page':page,'paragraph':i,'chunk':j,'type':'text'}]
                    )
            print("page:", page, "paragraphs:", len(paragraphs), "chunks:", total_chunks)

    
    def process_pdf(self, pdf_filename:str=None, pickle_filename: str='dicts_images_text.pickle', reload=False, images_dir='images/'):
        
        if (pdf_filename is not None) and (not reload):
            self.dict_mapping_pages = self._process_pdf_pages(pdf_filename)
            self.dict_extracted_paragraphs = self._extract_paragraphs(pdf_filename)
            with open(self.dir_path + pickle_filename, 'wb') as file:
                pickle.dump([self.dict_mapping_pages,self.dict_extracted_paragraphs],file)
        else:
            self.dict_mapping_pages, self.dict_extracted_paragraphs = pd.read_pickle(self.dir_path + pickle_filename)

        # Persist images in vector store
        self._ingest_images(images_dir)
        # Persist text in vector store
        self._ingest_text()

    
    def get_vector_store_collection(self):
        return self.chroma_collection
    
    def get_vector_store_client(self):
        return self.chroma_client

In [ ]:

# Folder with pdf and extracted images
path = "./input/"

dir_path = path
create = False # Set to True if ChromaDB needs to be re-created
pipeline = IngestionPipeline(dir_path=dir_path, create=create)
# pipeline.process_pdf(pdf_filename='photos.pdf')
# pipeline.process_pdf(pdf_filename='historiausina.pdf') # Uncomment to process the PDF with text and images

chroma_client = pipeline.get_vector_store_client()
chroma_collection = pipeline.get_vector_store_collection()


Updating vector store: ./input/chromadb/
Recovering existing collection ...
elements: 898


In [4]:
import base64
import io
from io import BytesIO

import numpy as np
from PIL import Image


def resize_base64_image(base64_string, size=(128, 128)):
    """
    Resize an image encoded as a Base64 string.

    Args:
    base64_string (str): Base64 string of the original image.
    size (tuple): Desired size of the image as (width, height).

    Returns:
    str: Base64 string of the resized image.
    """
    # Decode the Base64 string
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # Resize the image
    resized_img = img.resize(size, Image.LANCZOS)

    # Save the resized image to a bytes buffer
    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    # Encode the resized image to Base64
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def is_base64(s):
    """Check if a string is Base64 encoded"""
    try:
        return base64.b64encode(base64.b64decode(s)) == s.encode()
    except Exception:
        return False


def split_image_text_types(docs):
    """Split numpy array images and texts"""
    images = []
    text = []
    for doc in docs:
        doc = doc.page_content  # Extract Document contents
        if is_base64(doc):
            # Resize image to avoid OAI server error
            images.append(
                resize_base64_image(doc, size=(250, 250))
            )  # base64 encoded str
        else:
            text.append(doc)
    return {"images": images, "texts": text}

In [5]:
# TODO: This should be part of a RAG class or similar
def search(query: str, vector_store, k=3, kind=None, text_threshold=0.5, image_threshold=0.7):
    # Note: Processing pipeline must be run first!
    if kind is not None:
        retrieved = vector_store.query(query_texts=[query], include=['documents', 'metadatas','uris','distances'], n_results=k, where={"type": kind})
            # print(kind, len(retrieved))
    else:
        retrieved = vector_store.query(query_texts=[query], include=['documents', 'metadatas','uris','distances'], n_results=k)
        # print(len(retrieved))
        
    # TODO: thresholds are not verified!    
    return retrieved

In [6]:
# query = "En qué año se fundó la Usina?"
query = "Primer edificio de la Usina de Tandil"
results = search(query, chroma_collection, k=10, kind='image')
print(len(results))
results

8


{'ids': [['stitched_group_23_1',
   'stitched_group_38_0',
   'stitched_group_22_1',
   'stitched_group_23_2',
   'stitched_group_71_1',
   'stitched_group_18_0',
   'stitched_group_28_0',
   'stitched_group_31_1',
   'stitched_group_71_5',
   'stitched_group_47_1']],
 'embeddings': None,
 'documents': [[None, None, None, None, None, None, None, None, None, None]],
 'uris': [['./input/images/stitched_group_23_1.png',
   './input/images/stitched_group_38_0.png',
   './input/images/stitched_group_22_1.png',
   './input/images/stitched_group_23_2.png',
   './input/images/stitched_group_71_1.png',
   './input/images/stitched_group_18_0.png',
   './input/images/stitched_group_28_0.png',
   './input/images/stitched_group_31_1.png',
   './input/images/stitched_group_71_5.png',
   './input/images/stitched_group_47_1.png']],
 'data': None,
 'metadatas': [[{'page': 23, 'type': 'image'},
   {'page': 38, 'type': 'image'},
   {'page': 22, 'type': 'image'},
   {'page': 23, 'type': 'image'},
   {'pag

In [7]:
# chroma_collection.name
# chroma_client.list_collections()

In [8]:
vector_store_from_client = Chroma( # This is the Langchain wrapper
    client=chroma_client,
    collection_name=chroma_collection.name,
    embedding_function=OpenCLIPEmbeddings(model_name="ViT-B-32", checkpoint="laion2b_s34b_b79k")
)

In [9]:
# from typing import Optional
# from pydantic import BaseModel
# from langchain.chains.query_constructor.ir import (
#     Comparator,
#     Comparison,
#     Operation,
#     Operator,
#     StructuredQuery,
# )
# from langchain.retrievers.self_query.chroma import ChromaTranslator


# class Search(BaseModel):
#     query: str
#     kind: Optional[str]

# search_query = Search(query="type", kind="image")

# def construct_comparisons(query: Search):
#     comparisons = []
#     if query.kind is not None:
#         comparisons.append(
#             Comparison(
#                 comparator=Comparator.EQ,
#                 attribute="type",
#                 value=query.kind,
#             )
#         )
#     return comparisons

# comparisons = construct_comparisons(search_query)
# _filter = Operation(operator=Operator.AND, arguments=comparisons)
# filter_for_chroma = ChromaTranslator().visit_operation(_filter)
# filter_for_chroma

In [10]:
# Make retriever
retriever = vector_store_from_client.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.2, 'k': 10}, # "filter": {'type': {'$eq': 'image'}}}
)
# TODO: Maybe we need a custom retriever for the multimodal collection, interacting directly with ChromaDB for images
# https://python.langchain.com/docs/how_to/custom_retriever/

In [11]:
query = "carreta"
retriever.invoke(query) #, metadata={"type": 'image'})

[Document(id='paragraph_72_0_2', metadata={'chunk': 2, 'page': 72, 'paragraph': 0, 'type': 'text'}, page_content="Lo hicimos en un dia,en una caravana. Lo tinico que les pedi a los empleados fue que cada cual levara suspertenencias. Y cuando llegaron a Nigro se encontraron con sus oficinas y un cartelitodel escritorio que le tocaba a cada uno”,'” conté Cabitto.El ultimo acto como presidente de la Usina del Dr. Carlos Nicolini, el 2 de enerode 2008, fue para presidir la inauguracién del edificio con que la empresa dejépara siempre en el pasado el trauma de “las dos usinas”. Con la presidencia enciernes del Ing. Daniel"),
 Document(id='paragraph_44_0_3', metadata={'chunk': 3, 'page': 44, 'paragraph': 0, 'type': 'text'}, page_content='van y vuelven a lo largo de esta crénica.Uno de los primeros camiones de la Usina'),
 Document(id='paragraph_23_0_13', metadata={'chunk': 13, 'page': 23, 'paragraph': 0, 'type': 'text'}, page_content='del barrio de laEstacion contra otros vecinos que habiana

In [12]:
def prompt_func(data_dict):
    # Joining the context texts into a single string
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        image_message = {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{data_dict['context']['images']}"
            },
        }
        messages.append(image_message)

    # Adding the text message for analysis
    text_message = {
        "type": "text",
        "text": (
            "As an expert historian and particularly in Tandil history, your task is to analyze and interpret text and images, "
            "considering their historical and cultural significance. Provide your response in Spanish. Alongside the images, you will be "
            "provided with related text to offer context. Both will be retrieved from a vectorstore based "
            "on user-input keywords. Please use your extensive knowledge and analytical skills to provide a "
            "comprehensive summary that includes:\n"
            "- A detailed description of your answer.\n"
            "- The historical and cultural context for the text and images (if any).\n"
            "- An interpretation of the image's symbolism and meaning.\n"
            "- Connections between the text and images (if any).\n\n"
            f"User-provided keywords: {data_dict['question']}\n\n"
            "Text and / or tables:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)

    return [HumanMessage(content=messages)]

In [13]:
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv
from pathlib import Path

ENV_PATH = Path('.') / 'andres.env'
result = load_dotenv(dotenv_path=ENV_PATH.resolve(), override=True)
print("Reading OPENAI config:", ENV_PATH.resolve(), result)

# load_dotenv()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["OPENAI_MODEL_NAME"] = os.getenv("LLM_MODEL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

Reading OPENAI config: /Users/adiazpace/Documents/GitHub/openai-conversational-voice-chatbot/andres.env True


In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.environ["OPENAI_MODEL_NAME"],
    api_key=os.environ["OPENAI_API_KEY"],
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

model = ChatOpenAI(temperature=0, model=os.environ["OPENAI_MODEL_NAME"], api_key=os.environ["OPENAI_API_KEY"])

In [15]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Italian. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='Amo programmare.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 31, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-e059c81a-42f9-4f38-b13a-9ea9c7e2bf28-0', usage_metadata={'input_tokens': 31, 'output_tokens': 6, 'total_tokens': 37, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [16]:
# RAG pipeline
chain = (
    {
        "context": retriever | RunnableLambda(split_image_text_types),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(prompt_func)
    | model
    | StrOutputParser()
)

In [17]:
from IPython.display import HTML, display, Markdown


def plt_img_base64(img_base64):
    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'

    # Display the image by rendering the HTML
    display(HTML(image_html))


# query = "Origen de la Usina y su fundación"
query = "Primer edificio de la Usina de Tandil"
docs = retriever.invoke(query, k=20)
for doc in docs:
    if is_base64(doc.page_content):
        plt_img_base64(doc.page_content)
    else:
        print(doc.page_content)

Indgenes de la construccién del edificio tinico.que se le da a Usicom aludia en un principio hacia dénde querian orientarse esosnuevos negocios, tal como lo explicé el gerente Mario Cabitto: “Usicom se llamaUsicom porque era una sintesis de ‘Usina- Comunicaciones’, dado que se pensaba haceralgo en el tema vinculado a comunicacién como television, telefonia e internet. Es decircablear con fibra éptica y llegar a la comunidad con esos tres servicios. Es mds iniciamoslos trdmites con la Comision Nacional de
de nuestroedificio. La pluma brillante que dejo co-trer en el original los dicterios y piramida-les razonamientos que la venenosa hojitadifundiera, se escudaba entonces en elanénimo, hoy un boletin de ‘La Semana’,que lleva el ndmero 1 como advirtiendoque su aparicion no sera Gnica; por la re-peticion de iguales términos e idénticosrazonamientos, descubre al IGNORADOautor de aquél. Ya le conociamos el nido,pero al hombre le gustaba jugar a las es-condidas. Hoy ya habra llegado a una am-

In [18]:
response = chain.invoke(query)
display(Markdown(response))

### Análisis del Primer Edificio de la Usina de Tandil

#### Descripción Detallada
El texto proporcionado ofrece un panorama sobre la construcción del primer edificio de la Usina de Tandil, resaltando su importancia en el desarrollo tecnológico y social de la ciudad durante las décadas de 1930 y 1940. Se menciona que la Usina, inicialmente concebida como un centro de comunicaciones, se convirtió en un símbolo de progreso y modernización para la comunidad. La obra fue diseñada por la arquitecta Ruth Massera de Castelnuovo y se caracterizó por su estructura imponente, que se alza como un exponente del avance tecnológico en un contexto de creciente complejidad social.

#### Contexto Histórico y Cultural
La construcción de la Usina se sitúa en un período crucial para Tandil, donde la llegada de servicios básicos como la electricidad y el agua corriente transformó la vida cotidiana de sus habitantes. Entre 1930 y 1940, la ciudad experimentó un cambio significativo en su estructura social, con un ascenso de las clases medias y una fragmentación de la vida social. Este contexto se refleja en la necesidad de crear infraestructuras que respondieran a las demandas de una población en crecimiento y en transformación.

La Usina no solo representó un avance tecnológico, sino que también se convirtió en un símbolo de la identidad local, uniendo a la comunidad en torno a un proyecto común. La referencia a la "pluma brillante" y a la "febril barriada obrera" en el texto sugiere un reconocimiento del esfuerzo colectivo y la importancia del trabajo en equipo para lograr el progreso.

#### Interpretación de la Simbología de la Imagen
Aunque no se han proporcionado imágenes específicas, se puede inferir que las representaciones visuales del edificio de la Usina, como el hall de planta baja y el hall de planta alta, simbolizan la modernidad y el progreso. La arquitectura del edificio, con sus "líneas severas e imponentes", puede interpretarse como un reflejo de la seriedad y la determinación de la comunidad de Tandil para avanzar hacia un futuro más próspero.

El uso de materiales y el diseño funcional del edificio también pueden ser vistos como una metáfora de la resiliencia de la ciudad, que, a pesar de los desafíos, se esfuerza por construir un entorno que beneficie a todos sus habitantes.

#### Conexiones entre el Texto y las Imágenes
El texto y las imágenes se complementan al narrar la historia de la Usina como un proyecto que no solo se limita a la construcción física de un edificio, sino que también abarca el desarrollo de una comunidad. La mención de la participación de empresas locales y la presión de instituciones en el proceso de construcción subraya la interconexión entre el desarrollo económico y social de Tandil.

Además, el texto destaca la importancia de la participación ciudadana en la toma de decisiones, lo que se refleja en la elección de proyectos y en la construcción de un consenso en torno a la Usina. Esto sugiere que el edificio no solo es un espacio físico, sino un punto de encuentro para la comunidad, donde se forjan lazos y se construye identidad.

### Conclusión
El primer edificio de la Usina de Tandil es un testimonio del progreso tecnológico y social de la ciudad en un momento clave de su historia. A través de su construcción, se evidencia la transformación de la vida cotidiana de los tandilenses y la importancia de la colaboración comunitaria. Las imágenes del edificio, junto con el texto, ofrecen una rica narrativa que invita a reflexionar sobre el papel de la infraestructura en la construcción de la identidad y el desarrollo de una comunidad.